
<h1><center> Exemplary GeMSE Analysis </center></h1>

<br>
<br>
<br>
<br>
<br>

This jupyter notebook is used to showcase an exemplary GeMSE analysis utilizing function calls to Moritz von Sivers' analysis scripts.



<br>
<br>

#### Imports

<br>


In [1]:

import gemseana
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import subprocess
import json



<br>
<br>

#### Path Definitions

<br>


In [8]:

# stuff
gemse_mint = "#5a8fa3" # mint green color of the GeMSE logo


### paths: gemse analysis infrastructure
abspath_gemse_analysis_infrastructure = "/home/gemse_analysis_infrastructure/"
abspath_gemse_external_directory = abspath_gemse_analysis_infrastructure +"ex/"
# root
abspath_root = abspath_gemse_analysis_infrastructure +"root/root_v6.22.06.Linux-ubuntu18-x86_64-gcc7.5/root/"
pathstring_thisroot = abspath_root +"bin/thisroot.sh"
# bat
abspath_bat = abspath_gemse_analysis_infrastructure + "bat/BAT-0.9.4.1/"
abspath_bat_install = abspath_gemse_analysis_infrastructure + "bat_install/"
# moritz' scripts
abspath_gemse_root_scripts = abspath_gemse_analysis_infrastructure + "gemse_root_scripts/"
abspath_gemse_analysis = abspath_gemse_analysis_infrastructure + "gemse_analysis/"


# paths: raw data
abspath_gemse_measurements = abspath_gemse_external_directory +"/exemplary_gemse_measurements/"


# exemplary gemse analysis files
abspath_gemse_analysis_files = abspath_gemse_measurements +"gemse_analysis_files/"
abspath_gemse_analysis_calibration = abspath_gemse_analysis_files +"calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root"
abspath_gemse_analysis_background = abspath_gemse_analysis_files +"background_combined.root"
abspath_gemse_analysis_efficiency = abspath_gemse_analysis_files +"simulation/simulated_efficiencies.root"
abspath_gemse_analysis_resolution = abspath_gemse_analysis_files +"komar_gaon_ch00_full.dat.root_spectrum_calibrated_0-1188660s.root_resolution_function.root"
abspath_gemse_analysis_isotopes = abspath_gemse_analysis_files +"isotope_parameters/"


# paths: ptfesc
abspath_ptfesc = "/home/daniel/Desktop/arbeitsstuff/20180705__monxe/monxe_studies/ptfesc/"



<br>
<br>

#### Sanple Definitions

<br>


In [4]:

sample_id = "cuboid_04_lanza"
abspath_measurement_folder = abspath_gemse_measurements +"2020-12-10_cuboid_04_lanza/"
mca_list_files = [
    "20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt", # second file (the first one was empty due to crash)
    "20210102__cuboid_04_lanza__old_veto__list_file_ch000.txt", # third file (after second crash)
    "20210114__cuboid_04_lanza__old_veto__list_file_ch000_ch000.txt"] # fourth file (after third crash)



<br>
<br>

#### Rates

<br>

The cell below is used to plot the rate histograms for all the input mca list files. This is done to ensure that there is no malevolent data included (in the sense of too high rates, e.g., due to radon enclosure or a missing GN2 purge).

<br>


In [7]:

if [True, False][0]:

    # looping over all mca list files
    for pathstring_raw_data in [abspath_measurement_folder +i for i in mca_list_files]:

        # mca list file ---> root file
        gemseana.make_rootfile_list(
            input_pathstring_mca_list_file = pathstring_raw_data,
            input_pathstring_calibration_function = abspath_gemse_analysis_calibration,
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts)

        # root file ---> calibrated spectrum
        gemseana.make_spectrum_list(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_time_window = [0,0])

        # plot rate for energy window
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_energy_calibration = True,
            input_binwidth = 1800,
            input_pulse_height_range = [50,2700]) # energy window taken into account, in keV, cutting lowest and highest bins (noise and overflow)


make_rootfile_list(): executing
/home/gemse_analysis_infrastructure/gemse_root_scripts/make_rootfile_list /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt ./gemse_analysis_files/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root

make_spectrum_list(): executing
/home/gemse_analysis_infrastructure/gemse_root_scripts/make_spectrum_list --file /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt.root --energy

plot_rate(): executing
/home/gemse_analysis_infrastructure/gemse_root_scripts/plot_rate --file /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt.root --energy --range_min 50 --range_max 2700 --binwidth 1800

make_rootfile_list(): executing
/home/gemse_


<br>
<br>

#### Analysis

<br>

Executing the cell below will start the actual analysis chain, consisting of single calls to Moritz' analysis scripts.

<br>


In [9]:

if [True, False][0]:

    ### running the GeMSE analysis framework
    gemseana.all_in_one_gemse_analysis(
        input_pathstrings_mca_list_files = [abspath_measurement_folder +i for i in mca_list_files],
        input_time_windows = [
            [[0, 0]], # total: 1400380 s
            [[0, 0]], # total: 976128 s
            [[0, 0]]], # total: 324078 s
        input_pathstring_gemse_analysis_configuration_file = gemseana.gen_analysis_configuration_file(
            pathstring_output = abspath_measurement_folder +"analysis_configuration_file.txt",
            sample_name = sample_id,
            abspath_isotope_parameters_folder = abspath_gemse_analysis_isotopes,
            abspath_sample_spectrum_root_file = abspath_measurement_folder +"final_calibrated_added_spectrum.root",
            abspath_background_spectrum_root_file = abspath_gemse_analysis_background,
            abspath_efficiency_root_file = abspath_gemse_analysis_efficiency,
            abspath_resolution_root_file = abspath_gemse_analysis_resolution,
            abspath_results_folder = abspath_measurement_folder,
            accuracy_of_mcmc = ["low", "medium", "high"][0],
            threshold_on_bayes_factor = 0.33,
            cl_for_activity_limit = 0.95,
            fract_uncert_efficency = 0.02,
            list_isotopes_to_analyze = ["U238", "Ra226", "Th228", "Ra228", "Co60", "K40", "Cs137", "#Mn54", "#Ti44", "#Na22", "#Al26"]),
        input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
        input_abspath_gemse_analysis = abspath_gemse_analysis,
        input_pathstring_calibration_function = abspath_gemse_analysis_calibration)
        # writing the analysis results in wiki syntax to file
    gemseana.gen_analysis_results_wiki_syntax_file(
        pathstring_gemse_analysis_summary = abspath_measurement_folder +sample_id +"_activities_summary.txt",
        pathstring_gemse_analysis_summary_wiki_syntax = abspath_measurement_folder +sample_id +"_activities_summary_wikisyntax.txt")


all_in_one_gemse_analysis(): abspath_measurement_folder:
/home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/

#################################################################
all_in_one_gemse_analysis(): mca list file(s) ---> root file(s)
#################################################################

make_rootfile_list(): executing
/home/gemse_analysis_infrastructure/gemse_root_scripts/make_rootfile_list /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/gemse_analysis_files/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root

make_rootfile_list(): executing
/home/gemse_analysis_infrastructure/gemse_root_scripts/make_rootfile_list /home/gemse_analysis_infrastructure/ex//exemplary_gemse_measurements/2020-12-10_cuboid_04_lanza/20210102__cuboi